In [34]:
#tu rzeczy do instalacji, zaczynac od !
#%pip install tensorflow
#%pip install matplotlib
#%pip install pandas
#tu sklearn, możliwe że trzeba zrestartowac krenel po updacie
#%pip install -U scikit-learn

In [35]:
#tu biblioteki
import numpy as np
import pandas as pd
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras import regularizers
from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve, auc, accuracy_score
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

print("Import done")

Num GPUs Available:  0
Import done


In [36]:
#GPUs - tu poki co nie dziala, trzeba postawic srodowisko wirtualne
gpus = tf.config.experimental.list_physical_devices('GPU')
print (len(gpus), " GPUs found!")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), " Physical GPUs, ", len(logical_gpus), " Logical GPUs")
    except RuntimeError as e:
        print(e)
print("GPUs done")

0  GPUs found!
GPUs done


In [37]:
def directory_flow(train_directory, test_directory, val_directory, shape):
    #directory = nazwy folderow,
    #shape = rozmiar obrazka, u nas 256x256
    print ("generators start")
    image_generator=ImageDataGenerator()
    train_gen = image_generator.flow_from_directory(train_directory, target_size=shape, shuffle=True) #sciezka do train dir
    print ("train done")
    val_gen = image_generator.flow_from_directory(val_directory, target_size=shape, shuffle=True)
    print ("validate done")
    test_gen = image_generator.flow_from_directory(test_directory, target_size=shape, shuffle=True) #sciezka do test dir
    print ("generators done")
    #zastanowic sie nad zwracaniem validate gen
    return train_gen, test_gen, val_gen

In [38]:
#nie zadziala, bo jeszcze nie ma obrazkow
train_gen, test_gen, val_gen = directory_flow("./archive/TrainingData", "./archive/TestingData", "./archive/ValidationData", (224, 224))
print("Directory flow done!")

generators start
Found 30000 images belonging to 15 classes.
train done
Found 2000 images belonging to 15 classes.
validate done
Found 2407 images belonging to 15 classes.
generators done
Directory flow done!


In [39]:
# Do analizy danych
#labels = kategorie (rodzaje zwierzatek)
def plot_label_count(df, plot_title):
    '''
    This function take df and plot labels value counts
    '''

    # Define needed variables
    vcounts = df['labels'].value_counts()
    labels = vcounts.keys().tolist()
    values = vcounts.tolist()
    lcount = len(labels)

    if lcount > 55:
        print('The number of labels is > 55, no plot will be produced')

    else:
        plot_labels(lcount, labels, values, plot_title)

def plot_labels(lcount, labels, values, plot_title):
    width = lcount * 4
    width = np.min([width, 20])

    plt.figure(figsize= (width, 5))

    form = {'family': 'serif', 'color': 'blue', 'size': 25}
    sns.barplot(x=labels, y=values)
    plt.title(f'Images per Label in {plot_title} data', fontsize= 24, color= 'blue')
    plt.xticks(rotation= 90, fontsize= 18)
    plt.yticks(fontsize= 18)
    plt.xlabel('CLASS', fontdict= form)
    yaxis_label = 'IMAGE COUNT'
    plt.ylabel(yaxis_label, fontdict= form)

    rotation = 'vertical' if lcount >= 8 else 'horizontal'
    for i in range(lcount):
        plt.text(i, values[i] / 2, str(values[i]), fontsize= 12,
                rotation= rotation, color= 'yellow', ha= 'center')

    plt.show()

In [40]:
#Do analizy danych = przykładowe obrazki
def show_images(gen):
    '''
    This function take the data generator and show sample of the images
    '''

    # return classes , images to be displayed
    g_dict = gen.class_indices        # defines dictionary {'class': index}
    classes = list(g_dict.keys())     # defines list of dictionary's kays (classes), classes names : string
    images, labels = next(gen)        # get a batch size samples from the generator

    # calculate number of displayed samples
    length = len(labels)        # length of batch size
    sample = min(length, 25)    # check if sample less than 25 images

    plt.figure(figsize= (20, 20))

    for i in range(sample):
        plt.subplot(5, 5, i + 1)
        image = images[i] / 255       # scales data to range (0 - 255)
        plt.imshow(image)
        index = np.argmax(labels[i])  # get image index
        class_name = classes[index]   # get class of image
        plt.title(class_name, color= 'blue', fontsize= 12)
        plt.axis('off')
    plt.show()

In [41]:
#analiza danych - zadziala jak zamiast train_gen i val_gen da się dataframy
#pokaz ile obrazkow w kategorii

#plot_label_count(train_gen, 'train')
#plot_label_count(val_gen, 'validate')

In [42]:
#model
def make_model(shape):
    #te rzeczy w nawiasie zapobiegna przetrenowaniu
    base_model = tf.keras.applications.xception.Xception(include_top=False, weights="imagenet",input_shape=shape, pooling='max')
    model = Sequential([
        base_model,
        BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
        Dense(256, kernel_regularizer= regularizers.l2(l= 0.016), activity_regularizer= regularizers.l1(0.006),
                bias_regularizer= regularizers.l1(0.006), activation= 'relu'),
        Dropout(rate= 0.45, seed= 123),
        Dense(15, activation= 'softmax')
        ])
    #ograniczenie warstw zapobiegnie przetrenowaniu
    model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])
    return model
print("Model done!")

Model done!


In [43]:
model=make_model((224, 224, 3))
print("Model generated")

Model generated


In [44]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 2048)              20861480  
                                                                 
 batch_normalization_9 (Batc  (None, 2048)             8192      
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 256)               524544    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 15)                3855      
                                                                 
Total params: 21,398,071
Trainable params: 21,339,447
Non-trainable params: 58,624
_____________________________________

In [45]:
#zadziala jak wrzuce obrazki
model.fit(x=train_gen, epochs=40, validation_data=(val_gen),validation_steps= None, shuffle= False)

Epoch 1/40
938/938 [==============================] - 4732s 5s/step - loss: 3.3337 - accuracy: 0.9369 - val_loss: 0.9865 - val_accuracy: 0.9510
Epoch 2/40
938/938 [==============================] - 4551s 5s/step - loss: 0.5451 - accuracy: 0.9796 - val_loss: 0.4747 - val_accuracy: 0.9525
Epoch 3/40
513/938 [===============>..............] - ETA: 9:08:06 - loss: 0.3355 - accuracy: 0.9875

KeyboardInterrupt: 